<a href="https://colab.research.google.com/github/hyona-yu/HAR_python/blob/master/har_with2015uci_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import random

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
random.seed(42)
torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed_all(42)

In [ ]:
! git clone https://github.com/hyona-yu/Dataset.git

Cloning into 'Dataset'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 182 (delta 1), reused 19 (delta 1), pack-reused 161
Receiving objects: 100% (182/182), 133.20 MiB | 22.49 MiB/s, done.
Resolving deltas: 100% (118/118), done.
Checking out files: 100% (163/163), done.


In [ ]:
url = 'Dataset/HAPT Data Set/'

In [ ]:
feature_name_df = pd.read_csv(url + 'features.txt', sep='\s+', header=None, 
                              names=['column_name'])

feature_name = feature_name_df.values
train_csv = pd.read_csv(url + 'Train/X_train.txt', sep ='\s+', header = None)
train_csv.columns = feature_name
test_csv = pd.read_csv(url + 'Test/X_test.txt', sep = '\s+', header= None)
test_csv.columns = feature_name
train_y = pd.read_csv(url + 'Train/y_train.txt', header = None, names= ['label'])
test_y =pd.read_csv(url + 'Test/y_test.txt', header =None, names= ['label'])

In [ ]:
for i in range(len(test_y)):
  if test_y.loc[i,'label'] >=6 :
    test_csv.drop(i, inplace =True)
    test_y.drop(i, inplace = True)

In [ ]:
for i in range(len(train_y)):
  if train_y.loc[i,'label'] >=6 :
    train_csv.drop(i, inplace =True)
    train_y.drop(i, inplace = True)

#SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(random_state = 42)
params = {
    'C' : [0.5,1,10]
}
cv = GridSearchCV(clf, param_grid = params)
cv.fit(train_csv, train_y)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [0.5, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
pred = cv.predict(test_csv)

acc = accuracy_score(pred, test_y)
print("SVM :",acc)

SVM : 0.9551203590371277


In [ ]:
print(classification_report(pred, test_y))

              precision    recall  f1-score   support

           1       0.99      0.96      0.97       508
           2       0.97      0.95      0.96       482
           3       0.95      0.99      0.97       400
           4       0.90      0.97      0.93       468
           5       0.98      0.92      0.95       593

    accuracy                           0.96      2451
   macro avg       0.95      0.96      0.96      2451
weighted avg       0.96      0.96      0.96      2451



#CNN

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
# filter 수 늘리는게 중요? kernel size 크게
        self.layer1 = torch.nn.Sequential(torch.nn.Conv1d(input_size ,128, kernel_size= 1, stride= 1), torch.nn.MaxPool1d(kernel_size= 1), 
                                          torch.nn.ReLU()) #
        self.layer2 = torch.nn.Sequential(torch.nn.Conv1d(128,128, kernel_size= 1, stride= 1, padding =0),
                                          torch.nn.MaxPool1d(kernel_size= 1, stride = 1, padding = 0), 
                                          torch.nn.ReLU())

        self.layer3 = torch.nn.Linear(128 , output_size, bias = True) #(input_size - kernel_size + 2*padding_size)/stride + 1

        torch.nn.init.kaiming_uniform(self.layer3.weight)

    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.layer3(out)
        return out


In [ ]:
train_D = torch.FloatTensor(np.array(train_csv))
train_L = torch.FloatTensor(np.array(train_y))

In [ ]:
model = CNN(train_csv.shape[1], train_L.shape[1]).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  del sys.path[0]


In [ ]:
import torch.utils
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_D, train_L), batch_size = 100, shuffle= True, drop_last = True)

In [ ]:
running_loss =0
for e in range(1001):
  for i , data in enumerate(train_loader):
    x, y = data
    #print(x.shape)
    x = x.resize(100,561,1).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if (e+1) % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:358: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


[99] loss : 12.419
[199] loss : 4.868
[299] loss : 3.733
[399] loss : 3.175
[499] loss : 2.815
[599] loss : 2.564
[699] loss : 2.383
[799] loss : 2.232
[899] loss : 2.115
[999] loss : 2.004
fin


In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred=  model(test_D.view(-1,561,1).to(device))



In [ ]:
train_L

tensor([[5.],
        [5.],
        [5.],
        ...,
        [2.],
        [2.],
        [2.]])

In [ ]:
test_y.shape

(2451, 1)

In [ ]:
pred[2995]

IndexError: ignored

In [ ]:
acc = [np.round(pred[i])== np.array(test_y).reshape(len(test_csv),)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('CNN : ', real_acc)

CNN :  0.9163607


In [ ]:
print(classification_report(np.round(pred.cpu()), np.array(test_y)))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         9
         1.0       0.94      0.94      0.94       497
         2.0       0.93      0.85      0.89       518
         3.0       0.86      0.99      0.92       366
         4.0       0.86      0.97      0.91       451
         5.0       0.97      0.89      0.93       607
         6.0       0.00      0.00      0.00         3

    accuracy                           0.92      2451
   macro avg       0.65      0.66      0.66      2451
weighted avg       0.92      0.92      0.91      2451



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#LSTM

In [ ]:
class LSTM(torch.nn.Module):
  def __init__(self, n_class, bi):
    super(LSTM, self).__init__()

    self.lstm = torch.nn.LSTM(561, 128, 2,dropout = 0.1, bidirectional = bi)
    self.l1 = torch.nn.Linear(128, n_class)
    self.dropout = torch.nn.Dropout(0.1)
    self.tanh = torch.nn.Tanh()
    self.softmax = torch.nn.Softmax(dim = 1)
    self.h = torch.randn(2,1,128).to(device)
    self.c = torch.randn(2,1,128).to(device)
  def forward(self, x):
    #print(x.shape)
    x = x.permute(1,0,2)# 축 바꾸기. 전치행렬처럼
    #print(x.shape)

    out, hidden = self.lstm(x,(self.h,self.c) )
    #print(out.shape)
    out = self.tanh(out)
    out = self.dropout(out)
    out = out.contiguous().view(-1, 128)
    #print(out.shape)
    out = self.l1(out)
    #out = self.softmax(out)
    return out


In [ ]:
model = LSTM(train_L.shape[1], False).to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
running_loss =0
for e in range(1001):
  for i , data in enumerate(train_loader):
    x, y = data
    x = x.resize(1,100,561).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:358: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


[0] loss : 1.540
[100] loss : 3.151
[200] loss : 1.104
[300] loss : 0.648
[400] loss : 0.436
[500] loss : 0.342
[600] loss : 0.296
[700] loss : 0.277
[800] loss : 0.269
[900] loss : 0.264
[1000] loss : 0.251
fin


In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred_lstm = model(test_D.view(1,-1,561).to(device))


In [ ]:
acc = [np.round(pred[i])== np.array(test_y).reshape(len(test_csv),)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM : ', real_acc)

LSTM :  0.9163607


In [ ]:
print(classification_report(np.round(pred.cpu()), np.array(test_y)))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         9
         1.0       0.94      0.94      0.94       497
         2.0       0.93      0.85      0.89       518
         3.0       0.86      0.99      0.92       366
         4.0       0.86      0.97      0.91       451
         5.0       0.97      0.89      0.93       607
         6.0       0.00      0.00      0.00         3

    accuracy                           0.92      2451
   macro avg       0.65      0.66      0.66      2451
weighted avg       0.92      0.92      0.91      2451



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
